In [143]:
import os
import time

import pandas as pd

from linkedin_scraper import Person, actions, Company
from selenium import webdriver
from dotenv import load_dotenv

In [147]:
# スクレイピングに使うchromedriverを指定。google chromeのバージョンと合わせないとエラーが出るので、エラーが出たらインストールし直す
driver = webdriver.Chrome('./chromedriver')

In [148]:
# linkedinのログイン画面に遷移
driver.get('https://www.linkedin.com/login/ja?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')

In [149]:
load_dotenv("./.env")

USER_NAME = os.environ.get("USER_NAME")
PASSWORD = os.environ.get("PASSWORD")

In [150]:
# usernameのinput要素を指定
username = driver.find_element_by_id('username')


# usernameのinput要素にログインid(email)を入力
username.send_keys(USER_NAME)

# passwordのinput要素を指定
password = driver.find_element_by_id('password')

# passwordのinput要素にパスワードを入力
password.send_keys(PASSWORD)

# ログインボタンを指定
log_in_button = driver.find_element_by_xpath('//*[@type="submit"]')

# ログインボタンをクリック
log_in_button.click()

In [112]:
companies = [
    'mckinsey',
    'boston-consulting-group',
    'bain-and-company',
    'kearney',
    'rolandberger',
    'arthur-d--little',
    'dream-incubator',
    'corporate-directions-inc-',
    'strategyand',
    'monitordeloitte',
    'monitor-deloitte-ex-monitor-group',
    'accenture',
    'goldman-sachs',
    'morgan-stanley',
    'jpmorgan',
    'bank-of-america',
    'citi',
    'us-bank',
    'barclays-bank',
    'deutsche-bank',
    'credit-suisse',
    'bnp-paribas',
    'hsbc',
    'societe-generale'
]

In [125]:
# 企業ページのタレントページから日本に住んでいる人だけをリスティング
company = companies[2]
driver.get('https://www.linkedin.com/company/%s/people?keywords=日本' % company)


In [126]:
# 画面下にスクロールしたら自動でデータが追加されるので、自動スクロールのJavaScriptを記述し、実行
# time.sleep(5)でクローリングしていることを検知されない様にする
script = "window.scrollTo(0, document.body.offsetHeight - 2);"
for i in range(50):
    driver.execute_script(script)
    time.sleep(5)


In [127]:
# タレントごとのaタグを抽出
a_tags = driver.find_elements_by_class_name("link-without-visited-state")

In [129]:
# aタグからリンクを抽出
talent_list = [tag.get_attribute("href") for tag in a_tags]

In [130]:
# Dataframeに変換
df_list = pd.DataFrame({'talent': talent_list})

In [132]:
df_list.to_csv('./list/%s.csv' % company)

In [137]:
# 企業ごとに処理をまとめて実行
script = "window.scrollTo(0, document.body.offsetHeight - 2);"
for company in companies[3:]:
    driver.get('https://www.linkedin.com/company/%s/people?keywords=日本' % company)
    for i in range(50):
        driver.execute_script(script)
        time.sleep(5)
    a_tags = driver.find_elements_by_class_name("link-without-visited-state")
    talent_list = [tag.get_attribute("href") for tag in a_tags]
    df_list = pd.DataFrame({'talent': talent_list})
    df_list.to_csv('./list/%s.csv' % company)

In [139]:
import zipfile
from glob import glob

files = glob('./list/*')

with zipfile.ZipFile('./talent_list.zip', 'w', compression=zipfile.ZIP_DEFLATED) as new_zip:
    for file in files:
        new_zip.write(file)